In [211]:
import random
import os
import re
import numpy as np
from pprint import pprint
print(os.getcwd())
from Backend.py_utils import *
from Backend import perf_calc as perf_calc
from fastapi import FastAPI, Request
from fastapi.responses import RedirectResponse
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
from Backend import perf_calc as perf_calc
from Backend import rotation_calc as rotation_calc
from Backend import refusal as refusal
from database import database_handler
from fastapi.responses import JSONResponse
from Backend import handle_route as handle_route
from pydantic import BaseModel 
from Backend import wind_calc


g:\CompSciProject
g:\CompSciProject
g:\CompSciProject


In [10]:
def get_refusal_p3(uncorrected_ref_speed, rwy_slope): #runway slope

    TOP_FOLDER = "Backend/chart_dig/completed-takeoff/refusal-and-cef-speed"
    DIG_FILE_NAME = "refusal-runway-slope.dig"

    data = {}

    chart = ParseDig(f'./{TOP_FOLDER}/dig/{DIG_FILE_NAME}')
    for c in  chart.curveNames():
        yVector = [row [1] for row in chart.curve(c)]
        xVector = [row [0] for row in chart.curve(c)]
        scale_number = float(re.sub('[^0-9]','', c.replace("-", "_")))
        
        data[scale_number] = {
            "x": xVector,
            "y": yVector
        }
    
    runway_slope_scale = []
    result_slope = []

    for slope_scale, xy_pair_slope in data.items():
        xy_pair_slope = data[slope_scale]

        x_values = xy_pair_slope["x"]
        y_values = xy_pair_slope["y"]

        runway_slope_scale.append(slope_scale)
        result_slope.append(np.interp(rwy_slope,x_values, y_values))

    result = np.interp(uncorrected_ref_speed, runway_slope_scale, result_slope)

    return round(result)


In [60]:
get_refusal_p3(118, 0.4)

118

In [164]:
def get_refusal_p4(prev_data, wind_speed, tail_or_head):

    TOP_FOLDER = "Backend/chart_dig/completed-takeoff/refusal-and-cef-speed"
    DIG_FILE_NAME = "wind-velocity-for-refusal.dig"

    data = {}

    chart = ParseDig(f'./{TOP_FOLDER}/dig/{DIG_FILE_NAME}')
    for c in  chart.curveNames():
        yVector = [row [1] for row in chart.curve(c)]
        xVector = [row [0] for row in chart.curve(c)]
        scale_number = float(re.sub('[^0-9]','', c.replace("-", "_")))
 
        data[scale_number] = {
            "x": xVector,
            "y": yVector
        }

    if tail_or_head == "Headwind":
        wind_speed = wind_speed * 0.5 # apply a +50% factor to reported headwind
    elif tail_or_head == "Tailwind":
        wind_speed = wind_speed * 1.5 # apply a +150% factor to reported headwind
    else:
        wind_speed = wind_speed # do nothing if its a crosswind

    runway_windscale = []
    result_wind = []

    for windscale, xy_pair_wind in data.items():
        xy_pair_wind = data[windscale]

        x_values = xy_pair_wind["x"]
        y_values = xy_pair_wind["y"]

        runway_windscale.append(windscale)
        result_wind.append(np.interp(wind_speed, x_values, y_values))
    
    result = np.interp(prev_data, runway_windscale, result_wind)

    return round(result)

In [165]:
get_refusal_p4(118,20, "Headwind")

128

In [208]:
def get_refusal_p9(prev_data, yes_no):  

    TOP_FOLDER = "Backend/chart_dig/completed-takeoff/refusal-and-cef-speed"
    DIG_FILE_NAME = "refusal-anti-skid.dig"

    data = {}

    chart = ParseDig(f'./{TOP_FOLDER}/dig/{DIG_FILE_NAME}')
    for c in  chart.curveNames():
        yVector = [row [1] for row in chart.curve(c)]
        xVector = [row [0] for row in chart.curve(c)]
        scale_number = float(re.sub('[^0-9]','', c.replace("-", "_")))
        
        data[scale_number] = {
            "x": xVector,
            "y": yVector
        }

    if yes_no == True: # if the AS is operational
        yes_no = 0
    elif yes_no == False: # if the AS is not operational
        yes_no = 1
    else: yes_no = 0


    as_scale = []
    result_as = []

    for asscale, xy_pair_as in data.items():
        xy_pair_as = data[asscale]

        x_values = xy_pair_as["x"]
        y_values = xy_pair_as["y"]

        as_scale.append(asscale)
        result_as.append(np.interp(yes_no, x_values, y_values))

    result = np.interp(prev_data,as_scale,result_as)

    return round(result)


In [210]:
get_refusal_p9(115,False)

92

In [226]:
gwt = 145
get_to_factor = 2.7
get_rwy_available = 6
get_rwy_slope = 0.4
rsc = 0
rcr = 12
atcsoper = True
asoper = True
dragindex = 100
windspeed = 20
tail_or_head = "Headwind"

p1 = refusal.get_refusal_p1(get_to_factor, get_rwy_available)
p2 = refusal.get_refusal_p2(p1, gwt)
p3 = refusal.get_refusal_p3(p2, get_rwy_slope)
p4 = refusal.get_refusal_p4(p3, windspeed, tail_or_head)
p5 = refusal.get_refusal_p5(p4, dragindex)
p6 = refusal.get_refusal_p6(p5, rcr)
p7 = refusal.get_refusal_p7(p6, rsc)
p8 = refusal.get_refusal_p8(p7, atcsoper)
p9 = refusal.get_refusal_p9(p8, asoper)

print(p9)

p1 = 0
result p2 = 111
result p3 = 111
result p4 = 121
result p5 = 121
result p6 = 109
result p7 = 108
result p8 = 108
result p9 = 108
108


In [221]:
def get_refusal_p1(takeoff_factor, rwy_available): 
    
    TOP_FOLDER = "Backend/chart_dig/completed-takeoff/refusal-and-cef-speed"
    DIG_FILE_NAME = "refusal-step-1.dig" 

    data = {}

    chart = ParseDig(f'./{TOP_FOLDER}/dig/{DIG_FILE_NAME}')
    for c in  chart.curveNames():
        yVector = [row [1] for row in chart.curve(c)]
        xVector = [row [0] for row in chart.curve(c)]
        scale_number = float(re.sub('[^0-9]','', c.replace("-", "_")))
        
        data[scale_number] = {
            "x": xVector,
            "y": yVector
        }

    runway_scale_set = [] # runway scale (2,3,4, -> 14)
    result_basedon_to_factor_and_rwylen = [] # result for given takeoff factor scale based on runway length  

    for rwy_scale, xy_pair_for_rwy_avail in data.items():
        xy_pair_for_rwy_avail = data[rwy_scale]  #how long rwy is. Dictionary. dictionary_name[key] Key here is rwy_scale which is 2.0, 3.0, 4.0 etc

        x_values = xy_pair_for_rwy_avail["y"] # x is now y
        y_values = xy_pair_for_rwy_avail["x"] # y is now x

        runway_scale_set.append(rwy_scale)
        result_basedon_to_factor_and_rwylen.append(round(np.interp(takeoff_factor, x_values, y_values), 2))
    print(runway_scale_set)
    print(result_basedon_to_factor_and_rwylen)
    resultp1 = np.interp(rwy_available, runway_scale_set, result_basedon_to_factor_and_rwylen)
    print(f"p1 = {round(resultp1)}")
    return resultp1


In [224]:
get_refusal_p1(2, 2)

[2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0]
[np.float64(0.69), np.float64(0.54), np.float64(0.42), np.float64(0.32), np.float64(0.24), np.float64(0.15), np.float64(0.07), np.float64(-0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0)]
p1 = 1


np.float64(0.69)